In [3]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!sudo mkdir -p /usr/local/share/ca-certificates/Yandex/ && \
sudo wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt && \
sudo chmod 0655 /usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt


--2023-11-09 19:37:26--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt’

/usr/local/share/ca 100%[===================>]   3.50K  --.-KB/s    in 0s      

2023-11-09 19:37:26 (898 MB/s) - ‘/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt’ saved [3579/3579]



In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, StructField
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import udf
import os

In [3]:
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars org.apache.spark:spark-sql-kafka-0-10_2.12:2.0.0 --packages org.apache.spark:spark-sql-kafka-0-10_2.12:2.0.0 pyspark-shell'
# Создаем объект SparkSession, который является точкой входа в Spark SQL и Spark Streaming API.

spark = SparkSession\
    .builder\
    .appName("MySparkApp")\
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.3')\
    .getOrCreate()

In [13]:
options = {
    "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="mlops" password="mlops_pw";',
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol" : "SASL_SSL",
    "kafka.bootstrap.servers": 'rc1b-3mv6ns5irrql2lqs.mdb.yandexcloud.net:9091',
    "group.id": 'test_group',
    "subscribe": 'test',
}

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.sasl.jaas.config", 'org.apache.kafka.common.security.plain.PlainLoginModule required username="mlops" password="mlops_pw";') \
  .option("kafka.sasl.mechanism", "SCRAM-SHA-512") \
  .option("kafka.security.protocol", "SASL_SSL") \
  .option("kafka.bootstrap.servers", 'rc1b-3mv6ns5irrql2lqs.mdb.yandexcloud.net:9091') \
  .option("group.id", "test_group") \
  .option("subscribe", "test") \
  .load()

In [14]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [15]:
console = (df
           .writeStream
           .format('console'))

In [16]:
console.start().awaitTermination()

KeyboardInterrupt: 

In [15]:
pyspark.__version__

'3.0.3'